In [4]:
import cv2

# Load pre-trained DNN face detection model
modelFile = "res10_300x300_ssd_iter_140000.caffemodel"
configFile = "deploy.prototxt"
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)

# Open the default camera
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break

    (h, w) = frame.shape[:2]

    # Prepare image as input for the DNN model
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
                                 (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    # Loop through detected faces
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        # Only draw bounding box if confidence is above threshold
        if confidence > 0.6:  # you can increase to 0.7 or 0.8 if needed
            box = detections[0, 0, i, 3:7] * [w, h, w, h]
            (startX, startY, endX, endY) = box.astype("int")

            # Draw the face bounding box only (no confidence text)
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)

    # Show the output frame
    cv2.imshow('Face Detection', frame)

    # Press 'q' or 'Q' to quit
    key = cv2.waitKey(1) & 0xFF
    if key in [ord('q'), ord('Q')]:
        break

cap.release()
cv2.destroyAllWindows()